In [1]:
import sys
sys.path.append('../backend')

In [2]:
import modules.document.utils.policyqa_parser as policyqa_parser

policy_doc = policyqa_parser.parse_policy_text_by_title("ticketmaster.com", "../data/dev.json")
print(policy_doc.text)


Live Nation Entertainment Privacy Policy - Your Privacy Rights Effective July 20, 2012 (last updated October 08, 2013)
This Privacy Policy applies to the sites and apps where it appears. This Policy describes how we treat personal information we collect both online and offline. This includes on our websites or in our apps. It also includes at our box offices or in phone or email interactions you have with us. If you live in Canada, please read our Canadian Privacy Policy.
We collect information from and about you. Contact information. For example, we might collect your name and street address. We might also collect your phone number or email.
Payment and billing information. For example, we collect your credit card number and zip code when you buy a ticket.
Information you post. For example, we collect information you post in a public space on our website or on a third-party social media site.
Demographic information. We may collect information about events you like or products you bu

In [3]:
policyqa_parser.get_all_policy_titles("../data/dev.json")

['ticketmaster.com',
 'theatlantic.com',
 'sci-news.com',
 'yahoo.com',
 'style.com',
 'adweek.com',
 'cariboucoffee.com',
 'kaleidahealth.org',
 'fool.com',
 'buffalowildwings.com',
 'post-gazette.com',
 'internetbrands.com',
 'mlb.mlb.com',
 'eatchicken.com',
 'ted.com',
 'naturalnews.com',
 'cbsinteractive.com',
 'washingtonian.com',
 'dogbreedinfo.com',
 'walmart.com']

In [4]:
from config import config

config.add_title

True

In [5]:
from modules.document.service import create as create_document

async def save_policy_to_db(policy_doc):
    policy_obj = await create_document(policy_doc)
    return policy_obj

async def save_all_policies_to_db(file_path: str="../data/dev.json"):
    policy_titles = policyqa_parser.get_all_policy_titles(file_path)
    for title in policy_titles:
        policy_doc = policyqa_parser.parse_policy_text_by_title(title, file_path)
        policy_obj = await save_policy_to_db(policy_doc)
        print(f"saved policy id:{policy_obj.id}, title:{policy_obj.title}, doc_type:{policy_obj.doc_type} to database")

# policy_doc = policyqa_parser.parse_policy_text_by_title("rockstargames.com", "../data/test.json")
# policy_obj = await save_policy_to_db(policy_doc)
# print(f"saved policy id:{policy_obj.id}, title:{policy_obj.title}, doc_type:{policy_obj.doc_type} to database")

# Only run this to save all policies to database
# await save_all_policies_to_db("../data/dev.json")

In [6]:
from modules.answer.schemas import AnswerCreate
from modules.llm.llm_infos import Model

request = AnswerCreate(
    question="in ticketmaster.com, Does the company collect user's financial information?",
    model=Model.Gpt4,
    prompt="Answer the question by copying exactly a portion of the contexts, "
           "The portion you copy to answer the question should be as short and concise as possible, "
           "and if the answer is not contained within the text in the contexts, say `I don't know.`"
)


In [7]:
from modules.answer.service import create as create_answer
from collections import deque
import openai

openai.api_key = config.api_key
#question_embedding, answer_embeddings, answer_generator, num_tokens = await create_answer(request)
#answer_text = deque(answer_generator, maxlen=1)[0]

#print(answer_text)
#print(f'num_tokens: {num_tokens}')

In [8]:
from modules.llm.utils import vector_similarity
import pprint
    
def get_relevant_embeddings(answer_embeddings, question_embedding):
    relevant_embeddings = {"data": []}
    for i, embedding in enumerate(answer_embeddings):
        relevant_embedding = {}
        score = vector_similarity(embedding.values, question_embedding)
        relevant_embedding["embedding_id"] = str(embedding.id)
        relevant_embedding["rank"] = i+1
        relevant_embedding["title"] = embedding.document.title
        relevant_embedding["offset"] = embedding.offset
        relevant_embedding["score"] = score
        relevant_embedding["text"] = embedding.text
        relevant_embeddings["data"].append(relevant_embedding)
    return relevant_embeddings

#pprint.pprint(get_relevant_embeddings(answer_embeddings, question_embedding))
#print(get_relevant_embeddings(answer_embeddings, question_embedding))

In [9]:
print(f'The average context size is: {policyqa_parser.calculate_avg_context_size("../data/dev.json")}')
print(f'The average answer size is: {policyqa_parser.calculate_avg_answer_size("../data/dev.json")}')

The average context size is: 430.5452961672474
The average answer size is: 70.21095301125082


In [10]:
file_path = "../data/test.json"
file_name = file_path.split("/")[-1].split(".")[0]
file_name

'test'

In [11]:
import json
from tqdm import tqdm
import time

def count_qas(file_path: str="../data/dev.json"):
    count = 0
    with open(file_path, "r") as f:
        json_data = json.load(f)
        if "data" in json_data:
            for policy in tqdm(json_data["data"]):
                for paragraph in tqdm(policy["paragraphs"]):
                    for qa in paragraph["qas"]:
                        count += 1
                        time.sleep(0.001)
    return count


qa_count = count_qas("../data/dev.json")
print(f'total number of qas is: {qa_count}')

100%|██████████| 12/12 [00:00<00:00, 145.83it/s]

100%|██████████| 1/1 [00:00<00:00, 48.89it/s]

100%|██████████| 4/4 [00:00<00:00, 127.23it/s]

100%|██████████| 3/3 [00:00<00:00, 70.01it/s]

100%|██████████| 20/20 [00:04<00:00,  4.16it/s]

total number of qas is: 3809


In [12]:
import json
from tqdm import tqdm
import time
import logging
import os

rate_limit_per_minute = 500
tokens_per_minute = 10000
delay = 60.0 / rate_limit_per_minute

def get_saved_qas(file_path: str):
    saved_qas = []
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            json_data = json.load(f)
            if "data" in json_data:
                for qa in json_data["data"]:
                    saved_qas.append(qa["id"])
    return saved_qas

async def run_all_policy_qas(file_path: str="../data/dev.json", model: Model=Model.Gpt4):
    running_tokens = 0
    start_time = time.time()
    results = {"data": []}
    file_name = file_path.split("/")[-1].split(".")[0]
    
    # check if there are saved results and load them
    saved_qas = get_saved_qas(f"../results/{file_name}_results.json")
    if len(saved_qas) > 0:
        with open(f"../results/{file_name}_results.json", "r") as f:
            results = json.load(f)
        
    with open(file_path, "r") as f:
        json_data = json.load(f)
        if "data" in json_data:
            for policy in tqdm(json_data["data"]):
                title = policy["title"]
                logging.info(f"processing policy: {title}")
                for i, paragraph in tqdm(enumerate(policy["paragraphs"])):
                    for qa in paragraph["qas"]:
                        q_id = qa["id"]
                        if q_id in saved_qas:
                            print(f"skipping question {q_id} because it is already saved")
                            continue
                        question = qa["question"]
                        answers = qa["answers"]
                        request = AnswerCreate(
                            question=f'in {title}, {question}',
                            model=model,
                            prompt="Answer the question by copying exactly a portion of the contexts, "
                                   "The portion you copy to answer the question should be as short and concise as possible, "
                                   "and if the answer is not contained within the text in the contexts, say `I don't know.`"
                        )

                        if running_tokens > tokens_per_minute*0.8 and time.time() - start_time < 60.0:
                            time.sleep((time.time() - start_time)+15)
                            start_time = time.time()
                            running_tokens = 0
                        
                        question_embedding, answer_embeddings, answer_generator, num_tokens = await create_answer(request)
                        
                        running_tokens += num_tokens
                        
                        time.sleep(delay)
                        answer_text = deque(answer_generator, maxlen=1)[0]
                        relevant_embeddings = get_relevant_embeddings(answer_embeddings, question_embedding)
                        results["data"].append({
                            "title": title,
                            "id": q_id,
                            "question": question,
                            "answers": answers,
                            "pred_answer": answer_text,
                            "relevant_embeddings": relevant_embeddings["data"]
                        })
                    # save results to file after each paragraph
                    with open(f"../results/{file_name}_results.json", "w") as f:
                        json.dump(results, f, indent=4)
                        print(f"saved results to ../results/{file_name}_results.json for title {title} paragraph {i}")
                        
        
        
async def run_all_policy_qas_dev():
    await run_all_policy_qas("../data/dev.json")
    
async def run_all_policy_qas_test():
    await run_all_policy_qas("../data/test.json")

In [13]:
#test GPT-4 responses with small dataset

await run_all_policy_qas("../data/dev-small.json")

  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:05,  5.25s/it]

saved results to ../results/dev-small_results.json for title ticketmaster.com paragraph 0



2it [00:14,  7.44s/it]
100%|██████████| 1/1 [00:14<00:00, 14.87s/it]

saved results to ../results/dev-small_results.json for title ticketmaster.com paragraph 1


In [14]:
#get all responses for the dev dataset

#await run_all_policy_qas_dev()

In [15]:
from backend.evals.torchmetrics_eval_script import eval_squad_metrics

eval_squad_metrics("../results/dev_results.json")

100%|██████████| 3809/3809 [00:00<00:00, 236263.00it/s]


{'exact_match': tensor(1.9690), 'f1': tensor(17.8223)}

In [16]:
from backend.evals.squad_eval_script import Evaluator
import json

evaluator = Evaluator("../data/dev.json")
em, f1, precision, recall = evaluator.evaluate('../results/dev_results.json')
print(f"Exact Match: {em}\n F1-measure: {f1}\n Precision: {precision}\n Recall: {recall}")

Exact Match: 1.9690207403517983
 F1-measure: 17.992978969749082
 Precision: 15.164048089709462
 Recall: 50.71725889962931


In [17]:
from backend.evals.torchmetrics_eval_script import prepare_retrieval_data
top_k=2
_, indexes, preds, labels = prepare_retrieval_data("../results/dev_results.json", top_k=top_k)
print(len(indexes))
print(len(preds))
print(len(labels))

3809it [00:00, 243605.01it/s]

7618
7618
7618


In [18]:
print(indexes[0:15])
print(preds[0:15])
print(labels[0:100])

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7])
tensor([0.8960, 0.8597, 0.8611, 0.8609, 0.8517, 0.8396, 0.8565, 0.8545, 0.8570,
        0.8496, 0.8649, 0.8593, 0.8574, 0.8493, 0.8716])
tensor([False, False,  True, False, False, False,  True, False, False,  True,
         True, False,  True,  True, False,  True, False, False, False,  True,
        False, False,  True,  True, False,  True, False, False, False,  True,
        False, False,  True, False, False,  True,  True,  True, False,  True,
        False, False,  True,  True, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False, False,
         True, False,  True, False,  True, False,  True, False, False,  True,
         True, False,  True, False,  True,  True,  True, False, False, False,
         True, False,  True, False,  True, False, False, False,  True, False,
        False,  True,  True, False, False,  True,  True, False,  True, False])


In [19]:
from backend.evals.torchmetrics_eval_script import eval_retrieval_metrics
from tqdm import tqdm
import logging

# logging.disable(logging.INFO)

retrieval_metrics = []
top_k=5
for i in tqdm(range(1, top_k+1)):
    retrieval_metrics.append(eval_retrieval_metrics("../results/dev_results.json", top_k=i))

for retrieval_metrics in retrieval_metrics:
    print(f'{retrieval_metrics}')

  0%|          | 0/5 [00:00<?, ?it/s]
3809it [00:00, 221340.06it/s]

100%|██████████| 3809/3809 [00:02<00:00, 1768.01it/s]
C:\Users\onan\IdeaProjects\ai4esg_experiments_new\backend\evals\torchmetrics_eval_script.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  relevant_embeddings_ratio = torch.tensor(sum(targets) / len(targets))
 20%|██        | 1/5 [00:02<00:09,  2.48s/it]
3809it [00:00, ?it/s]

 40%|████      | 2/5 [00:05<00:07,  2.54s/it]
3809it [00:00, 245062.34it/s]

 60%|██████    | 3/5 [00:07<00:05,  2.64s/it]
3809it [00:00, 457322.49it/s]

 80%|████████  | 4/5 [00:10<00:02,  2.70s/it]
3809it [00:00, 243742.53it/s]

100%|██████████| 5/5 [00:13<00:00,  2.67s/it]

{'map@1': tensor(0.3437), 'mrr@1': tensor(0.3437), '%Answers found@1': tensor(0.3437), '%Relevant embeddings@1': tensor(0.3437)}
{'map@2': tensor(0.4056), 'mrr@2': tensor(0.4056), '%Answers found@2': tensor(0.4676), '%Relevant embeddings@2': tensor(0.3066)}
{'map@3': tensor(0.4267), 'mrr@3': tensor(0.4320), '%Answers found@3': tensor(0.5466), '%Relevant embeddings@3': tensor(0.2854)}
{'map@4': tensor(0.4363), 'mrr@4': tensor(0.4468), '%Answers found@4': tensor(0.6059), '%Relevant embeddings@4': tensor(0.2707)}
{'map@5': tensor(0.4416), 'mrr@5': tensor(0.4563), '%Answers found@5': tensor(0.6535), '%Relevant embeddings@5': tensor(0.2587)}


In [20]:
from backend.evals.torchmetrics_eval_script import eval_retrieval_metrics
top_k=5
retrieval_metrics = eval_retrieval_metrics("../results/dev_results.json", top_k=top_k)
print(f'The retrieval metrics @{top_k} are {retrieval_metrics}')

3809it [00:00, 220923.79it/s]
100%|██████████| 3809/3809 [00:02<00:00, 1713.16it/s]


The retrieval metrics @5 are {'map@5': tensor(0.4416), 'mrr@5': tensor(0.4563), '%Answers found@5': tensor(0.6535), '%Relevant embeddings@5': tensor(0.2587)}


In [21]:
import modules.document.utils.policyqa_parser as policyqa_parser

num_answers = policyqa_parser.count_answers("../data/dev.json")
num_answers

6044

In [22]:
from backend.modules.document.utils.policyqa_parser import count_questions

print(count_questions("../data/dev.json"))

3809
